# Import Libraries and Data

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn import linear_model
import torch
import torch.nn as nn
from tqdm import tqdm
import warnings

from sklearn.metrics import roc_auc_score

from sklearn.exceptions import ConvergenceWarning

In [2]:
DATA_PATH = '../data'
MODEL_PATH = '../src/models'

In [3]:
feature_matrix = pd.read_csv(f'{DATA_PATH}/processed/feature_matrix.csv')
feature_matrix.head()

,FPF_TARGET,minbalance,maxbalance,stdbalance,total_balance,avg_monthly_spending,num_monthly_purchase,memo_clean,unsufficient_balance,account_count,...,OVERDRAFT_prop,PAYCHECK_prop,PETS_prop,RENT_prop,SELF_TRANSFER_prop,TAX_prop,TRAVEL_prop,UNCATEGORIZED_prop,Overdraft Count,cum_weighted_def_val
0,0.0,0.52,253.01,125.590741,857.150161,5531.648836,79.521124,2.0,5,5,...,0.000000,0.0,0.000498,0.0,0.142151,0.000000,0.000830,0.089842,0.0,0.000000
1,0.0,6.11,31564.64,13215.093678,43016.714011,64529.528960,160.211203,0.0,4,4,...,0.000000,0.0,0.000000,0.0,0.016751,0.001093,0.004267,0.246412,0.0,0.000000
2,0.0,1092.82,31229.66,17278.364208,14667.666944,16997.552350,92.855324,0.0,3,3,...,0.000000,0.0,0.000000,0.0,0.261025,0.000000,0.002219,0.001825,0.0,0.000000
3,0.0,40.45,6425.61,4514.989935,-4188.933058,9778.291345,95.104454,0.0,2,2,...,0.000000,0.0,0.011885,0.0,0.000000,0.000000,0.000142,0.016086,0.0,0.000000
4,0.0,0.40,1009.99,438.528721,-2630.480387,16837.676865,119.566068,0.0,4,4,...,0.057938,0.0,0.001095,0.0,0.000000,0.000000,0.016481,0.140562,1.0,318.890041


# Define Functions
### Utility Functions

In [4]:
def train_valid_test_split(X, y, TEST_SIZE=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(
        X, 
        y, 
        test_size = TEST_SIZE, 
        random_state=random_state
    )

    X_train, X_valid, y_train, y_valid = train_test_split(
        X_train, 
        y_train, 
        test_size = TEST_SIZE, 
        random_state=random_state
    )

    return X_train, X_valid, X_test, y_train, y_valid, y_test

In [5]:
def get_metrics(pred, actual, display=True):
    cm = confusion_matrix(actual, pred)
    tn, fp, fn, tp = cm.ravel()

    TPR = tp / (tp + fn)
    FPR = tn / (tn + fp)
    roc_auc = roc_auc_score(actual, pred)
    if (display):
        print(f"Accuracy: {np.mean(pred == actual):.4f}")
        print(f"tn:  {tn},  fp:  {fp},  fn:  {fn}, tp:  {tp}")
        print(f"Precision:\t\t\t{tp / (tp + fp):.5f}")
        print(f"Recall:\t\t\t\t{tp / (tp + fn):.5f}")
        print(f"TPR:\t\t\t\t{TPR:.5f}")
        print(f"FPR:\t\t\t\t{FPR:.5f}")
        print(f"Positive likelihood ratio:\t{TPR/FPR:.5f}")
        print(f"Negative likelihood ratio:\t{FPR/TPR:.5f}")
        print(f"ROC-AUC Score:\t{roc_auc:.5f}")
        return
    return [tn, fp, fn, tp, TPR, FPR, TPR/FPR, FPR/TPR, roc_auc]

### Sequential Neural Net Functions

In [6]:
def train_SNN(
        model, 
        loss_fn, 
        optimizer, 
        X_train_tensor, 
        y_train_tensor, 
        X_valid_tensor, 
        y_valid_tensor, 
        n_epochs=300,
        batch_size=16, 
        display=True):
    
    batch_start = torch.arange(0, X_train_tensor.shape[0], batch_size)
    train_history = []
    valid_history = []
    best_valid = np.inf
    
    # training loop
    bar = tqdm(range(n_epochs))
    for epoch in bar:
        model.train()
        bar.set_description(f"Epoch {epoch+1}")
        for start in batch_start:
            # take a batch
            X_batch = X_train_tensor[start:start+batch_size]
            y_batch = y_train_tensor[start:start+batch_size]
            # forward pass
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            # update weights
            optimizer.step()
        # evaluate accuracy at end of each epoch
        model.eval()
        y_pred_train = model(X_train_tensor)
        y_pred_valid = model(X_valid_tensor)
        train_loss = float(loss_fn(y_pred_train, y_train_tensor))
        valid_loss = float(loss_fn(y_pred_valid, y_valid_tensor))
        
        bar.set_postfix(Train_MSE = float(train_loss), Valid_MSE = float(valid_loss))
        train_history.append(train_loss)
        valid_history.append(valid_loss)

        if (valid_loss < best_valid):
            torch.save(model.state_dict(), f'{MODEL_PATH}/nn_base_params.pt')

    if display:
        print(f"Best Epoch: {np.argmin(valid_history)}")
        print(f"Loss: {np.min(valid_history)}")
    model.load_state_dict(torch.load(f'{MODEL_PATH}/nn_base_params.pt'))
    return model, train_history, valid_history

In [7]:
def find_best_prob(pred_prob, actual, display=True):
    best_prob = 0
    best_ratio = np.inf
    prob_interval = 0.05
    with np.errstate(divide='ignore'):
        if display:
            print(
                f"i\t" + 
                f"ratio\t" + 
                f"roc_auc\t" + 
                f"acc\t" + 
                f"conf_matrix" + 
                "\n==================================================="
            )
        for i in np.arange(0, 1.0, prob_interval):
            pred = pred_prob > i
            metrics = get_metrics(pred, actual, display=False)
            
            PLR = metrics[6]
            NLR = metrics[7]
            roc_auc = metrics[8]
            ratio = (PLR + NLR)/2

            if display:
                print(
                    f"{i:.2f}\t" + 
                    f"{ratio:0.3f}\t" + 
                    f"{roc_auc:0.3f}\t" + 
                    f"{((metrics[0] + metrics[3]) / np.sum(metrics[:4]) * 100):.2f}\t" + 
                    f"{metrics[:4]}"
                )

            if (ratio < best_ratio):
                best_ratio = ratio
                best_prob = i
    if display: print(f"Best Threshold: {best_prob:0.2f}")
    return best_prob

# Randomize Features and Evaluate Impact

In [8]:
X = feature_matrix.iloc[:,1:].to_numpy()
y = feature_matrix.iloc[:,0].to_numpy()

In [9]:
feature_names = feature_matrix.iloc[:,1:].columns
log_reg_feature_scores = []
SNN_feature_scores = []

for i in range(X.shape[1]):
    print(f"Evaluating Feature {i+1}/{len(feature_names)}")
    # Randomize feature
    orig_feature = X[:,i]
    np.random.shuffle(X[:,i])

    # Create New Datasets
    X_train, X_valid, X_test, y_train, y_valid, y_test = train_valid_test_split(X, y, random_state=42)

    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
    X_valid_tensor = torch.tensor(X_valid, dtype=torch.float32)
    y_valid_tensor = torch.tensor(y_valid, dtype=torch.float32).reshape(-1, 1)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)
    
    # Define and Train Models
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ConvergenceWarning)
        # Define Logistic Regression model
        lin_model = linear_model.LogisticRegression(max_iter=1000)
        # Train Logistic Regression model
        lin_model.fit(X_train, y_train)

    # Define Sequential NN model
    model = nn.Sequential(
        nn.Linear(X.shape[1], 12),
        nn.ReLU(),
        nn.Linear(12, 24),
        nn.ReLU(),
        nn.Linear(24, 24),
        nn.ReLU(),
        nn.Linear(24, 12),
        nn.ReLU(),
        nn.Linear(12, 6),
        nn.ReLU(),
        nn.Linear(6, 1),
        nn.Sigmoid()
    )
    # loss function and optimizer
    loss_fn = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
    # Train Sequential NN model
    model, train_history, valid_history = train_SNN(
        model, 
        loss_fn,
        optimizer,
        X_train_tensor, 
        y_train_tensor, 
        X_valid_tensor, 
        y_valid_tensor, 
        n_epochs = 200,
        batch_size = 64,
        display=False
    )
    
    # Evaluate Logistic Rergression model
    test_pred = lin_model.predict(X_test)
    lin_reg_score = get_metrics(test_pred, y_test, display=False)[-1]
    
    # Evaluate Sequential NN model    
    # Find optimal threshold
    valid_pred_prob = model(X_valid_tensor).detach().numpy()[:,0]
    threshold = find_best_prob(valid_pred_prob, y_valid, display=False)
    # Find test roc_auc score
    test_pred_prob = model(X_test_tensor).detach().numpy()[:,0]
    test_pred = test_pred_prob > threshold
    SNN_score = get_metrics(test_pred, y_test, display=False)[-1]
    
    # Save evaluated scores
    log_reg_feature_scores.append(lin_reg_score)
    SNN_feature_scores.append(SNN_score)
    
    # Restore orig Dataset
    X[:,i] = orig_feature


Evaluating Feature 1/240


Epoch 200: 100%|██████████| 200/200 [00:13<00:00, 15.12it/s, Train_MSE=0.157, Valid_MSE=0.817]


Evaluating Feature 2/240


Epoch 200: 100%|██████████| 200/200 [00:13<00:00, 15.23it/s, Train_MSE=0.177, Valid_MSE=0.727]


Evaluating Feature 3/240


...

Epoch 200: 100%|██████████| 200/200 [00:13<00:00, 15.00it/s, Train_MSE=0.212, Valid_MSE=1.6]  


Evaluating Feature 238/240


Epoch 200: 100%|██████████| 200/200 [00:12<00:00, 15.41it/s, Train_MSE=0.218, Valid_MSE=1.48] 


Evaluating Feature 239/240


Epoch 200: 100%|██████████| 200/200 [00:13<00:00, 15.12it/s, Train_MSE=0.314, Valid_MSE=1.61]


## Final Feature Scores

In [23]:
feature_names = feature_matrix.iloc[:,1:].columns

In [35]:
SNN_sorted_scores = {
    k:v for k,v in sorted(
        dict(zip(feature_names, SNN_feature_scores)).items(), key=lambda x: x[1]
    )
}

log_reg_sorted_scores = {
    k:v for k,v in sorted(
        dict(zip(feature_names, log_reg_feature_scores)).items(), key=lambda x: x[1]
    )
}

In [38]:
# Top 20 features for Logistic Regression model
np.array(list(log_reg_sorted_scores.keys()))[:20]

array(['CHILD_DEPENDENTS_mean', 'MORTGAGE_mean', 'minCHILD_DEPENDENTS',
       'RENT_mean', 'PAYCHECK_mean', 'INSURANCE_prop', 'PETS_mean',
       'TRAVEL_mean', 'HEALTHCARE_MEDICAL_mean', 'BNPL_mean',
       'OVERDRAFT_mean', 'INSURANCE_mean', 'CREDIT_CARD_PAYMENT_mean',
       'CREDIT_CARD_PAYMENT_prop', 'PETS_prop', 'SELF_TRANSFER_mean',
       'TAX_mean', 'AUTOMOTIVE_mean', 'ATM_CASH_prop', 'EDUCATION_prop'],
      dtype='<U25')

In [39]:
# Top 20 features for SNN model
np.array(list(SNN_sorted_scores.keys()))[:20]

array(['MORTGAGE_prop', 'PETS_prop', 'GIFTS_DONATIONS_prop',
       'ACCOUNT_FEES_prop', 'SELF_TRANSFER_prop',
       'HEALTHCARE_MEDICAL_prop', 'EDUCATION_prop',
       'CHILD_DEPENDENTS_prop', 'cum_weighted_def_val', 'GROCERIES_prop',
       'OVERDRAFT_prop', 'ESSENTIAL_SERVICES_count',
       'FOOD_AND_BEVERAGES_prop', 'UNCATEGORIZED_prop', 'Overdraft Count',
       'ESSENTIAL_SERVICES_mean', 'PAYCHECK_prop', 'LOAN_prop',
       'CREDIT_CARD_PAYMENT_prop', 'ESSENTIAL_SERVICES_prop'],
      dtype='<U25')